# Data PreProcessing

# Import Data

In [ ]:
pip install Sastrawi
pip install swifter

In [ ]:
# pnkt tokenizing
# nltk.download('punkt')
# nltk.download('stopwords')

In [3]:
import csv
import re
import nltk
import Sastrawi
import sklearn
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np

In [4]:
df = pd.read_csv("dataset/raw_tweets.csv")
df

,user,date,renderedContent
0,abadi11311049,2022-12-30 23:50:34+00:00,@KAI121 @gueduluan Kalau penumpang baru saja m...
1,KtimunLaut,2022-12-30 23:40:33+00:00,"@beh1ndyouu Paling wajib scan QR, artinya waji..."
2,Dio_Nzee,2022-12-30 22:56:07+00:00,"@PinkyStevanie Semoga aja Pink, krn yg udah2 b..."
3,j0s3ph11,2022-12-30 22:39:36+00:00,@KAI121 jadi jika belum dapat vaksin booster b...
4,jasa_vaksin_,2022-12-30 22:15:30+00:00,"Ready tembak vaksin\r\nVaksin 1,2,3\r\nHapus d..."
...,...,...,...
7995,elshinta_tegal,2022-10-06 12:34:00+00:00,Kemenkes relokasi vaksin booster untuk atasi k...
7996,RadenRoro93,2022-10-06 12:32:50+00:00,Ceci Christele Plastic girl Unair&amp;Yashmin ...
7997,ngaca_aja,2022-10-06 12:30:55+00:00,"@KAI121 Kak admin,ada gak sih vaksin booster d..."
7998,yesungiedf,2022-10-06 12:28:41+00:00,@hereforsuju_ @AllRiseSilver @donghae861015 si...


# Cleansing Data

In [5]:
import string 
import re #regex library

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    
    # remove mention @
    text = re.sub(r"[@][\w_-]+"," ", text)
    
    # remove link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
df['cleansing'] = df['renderedContent'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

df['cleansing'] = df['cleansing'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

df['cleansing'] = df['cleansing'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

df['cleansing'] = df['cleansing'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

df['cleansing'] = df['cleansing'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

df['cleansing'] = df['cleansing'].apply(remove_singl_char)

# remove duplicate
df.drop_duplicates(subset=['cleansing'], inplace=True)

df.loc[0:30,['renderedContent', 'cleansing']]

,renderedContent,cleansing
0,@KAI121 @gueduluan Kalau penumpang baru saja m...,Kalau penumpang baru saja masuk umur dan di ap...
1,"@beh1ndyouu Paling wajib scan QR, artinya waji...",Paling wajib scan QR artinya wajib vaksin boos...
2,"@PinkyStevanie Semoga aja Pink, krn yg udah2 b...",Semoga aja Pink krn yg udah biasa kebalikannya...
3,@KAI121 jadi jika belum dapat vaksin booster b...,jadi jika belum dapat vaksin booster blom bisa...
4,"Ready tembak vaksin\r\nVaksin 1,2,3\r\nHapus d...",Ready tembak vaksin Vaksin Hapus data Dll Pros...
5,@bobbynasution_ @pemko_medan Pak \r\nMohon inf...,Pak Mohon info RS di medan yang bisa vaksin bo...
6,@Noirditory Jangan takut aku udah vaksin booster,Jangan takut aku udah vaksin booster
7,Sebenarnya yang masuk rumah sakit dan meningga...,Sebenarnya yang masuk rumah sakit dan meningga...
8,@JohanDollah @drmusanordin saya dah 2 kali ken...,saya dah kali kena covid walaupun dah vaksinbo...
9,Mau tanya dong Kalo mau nonton konser harus va...,Mau tanya dong Kalo mau nonton konser harus va...


In [4]:
df.shape

(7302, 4)

# Case Folding

In [5]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
df['case_folding'] = df['cleansing'].str.lower()

df.loc[0:9,['cleansing', 'case_folding']]

,cleansing,case_folding
0,Kalau penumpang baru saja masuk umur dan di ap...,kalau penumpang baru saja masuk umur dan di ap...
1,Paling wajib scan QR artinya wajib vaksin boos...,paling wajib scan qr artinya wajib vaksin boos...
2,Semoga aja Pink krn yg udah biasa kebalikannya...,semoga aja pink krn yg udah biasa kebalikannya...
3,jadi jika belum dapat vaksin booster blom bisa...,jadi jika belum dapat vaksin booster blom bisa...
4,Ready tembak vaksin Vaksin Hapus data Dll Pros...,ready tembak vaksin vaksin hapus data dll pros...
5,Pak Mohon info RS di medan yang bisa vaksin bo...,pak mohon info rs di medan yang bisa vaksin bo...
6,Jangan takut aku udah vaksin booster,jangan takut aku udah vaksin booster
7,Sebenarnya yang masuk rumah sakit dan meningga...,sebenarnya yang masuk rumah sakit dan meningga...
8,saya dah kali kena covid walaupun dah vaksinbo...,saya dah kali kena covid walaupun dah vaksinbo...
9,Mau tanya dong Kalo mau nonton konser harus va...,mau tanya dong kalo mau nonton konser harus va...


# Tokenizing

In [6]:
# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing ---------

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df['tokenize'] = df['case_folding'].apply(word_tokenize_wrapper)

df.loc[0:9,['case_folding', 'tokenize']]

,case_folding,tokenize
0,kalau penumpang baru saja masuk umur dan di ap...,"[kalau, penumpang, baru, saja, masuk, umur, da..."
1,paling wajib scan qr artinya wajib vaksin boos...,"[paling, wajib, scan, qr, artinya, wajib, vaks..."
2,semoga aja pink krn yg udah biasa kebalikannya...,"[semoga, aja, pink, krn, yg, udah, biasa, keba..."
3,jadi jika belum dapat vaksin booster blom bisa...,"[jadi, jika, belum, dapat, vaksin, booster, bl..."
4,ready tembak vaksin vaksin hapus data dll pros...,"[ready, tembak, vaksin, vaksin, hapus, data, d..."
5,pak mohon info rs di medan yang bisa vaksin bo...,"[pak, mohon, info, rs, di, medan, yang, bisa, ..."
6,jangan takut aku udah vaksin booster,"[jangan, takut, aku, udah, vaksin, booster]"
7,sebenarnya yang masuk rumah sakit dan meningga...,"[sebenarnya, yang, masuk, rumah, sakit, dan, m..."
8,saya dah kali kena covid walaupun dah vaksinbo...,"[saya, dah, kali, kena, covid, walaupun, dah, ..."
9,mau tanya dong kalo mau nonton konser harus va...,"[mau, tanya, dong, kalo, mau, nonton, konser, ..."


# Normalisasi

In [7]:
normalized_word = pd.read_csv("kamus/normalisasi.csv")

normalized_word_dict = {}

for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]

df['normalisasi'] = df['tokenize'].apply(normalized_term)

df.loc[0:20,['tokenize', 'normalisasi']]

,tokenize,normalisasi
0,"[kalau, penumpang, baru, saja, masuk, umur, da...","[kalau, penumpang, baru, saja, masuk, umur, da..."
1,"[paling, wajib, scan, qr, artinya, wajib, vaks...","[paling, wajib, scan, qr, artinya, wajib, vaks..."
2,"[semoga, aja, pink, krn, yg, udah, biasa, keba...","[semoga, saja, pink, karena, yang, sudah, bias..."
3,"[jadi, jika, belum, dapat, vaksin, booster, bl...","[jadi, jika, belum, dapat, vaksin, booster, bl..."
4,"[ready, tembak, vaksin, vaksin, hapus, data, d...","[ready, tembak, vaksin, vaksin, hapus, data, d..."
5,"[pak, mohon, info, rs, di, medan, yang, bisa, ...","[pak, mohon, informasi, rs, di, medan, yang, b..."
6,"[jangan, takut, aku, udah, vaksin, booster]","[jangan, takut, aku, sudah, vaksin, booster]"
7,"[sebenarnya, yang, masuk, rumah, sakit, dan, m...","[sebenarnya, yang, masuk, rumah, sakit, dan, m..."
8,"[saya, dah, kali, kena, covid, walaupun, dah, ...","[saya, deh, kali, kena, covid, walaupun, deh, ..."
9,"[mau, tanya, dong, kalo, mau, nonton, konser, ...","[mau, tanya, dong, kalau, mau, nonton, konser,..."


# Filtering/Stopwords Removal

In [8]:
from nltk.corpus import stopwords
# nltk.download('stopwords')

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')

# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 'nyg', 'hehe', 'pen', 'pgn', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah', 'ada', 'adalah', 'adlah', 'adlh', 'agak', 'agar', 'agung', 'aja', 'akan', 'anda', 
                       'anu', 'apakah', 'apa', 'april', 'antara', 'atau','amat', 'apalagi', 'angin', 'api', 'akbp', 'azis', 
                       'andriansyah', 's', 'h', 'i', 'k', 'm', 'babhinkamtibmas', 'babinsa', 'bagaimanapun', 'bagi', 'bahwa', 
                       'bandara', 'bandung', 'beberapa', 'bbrp', 'bisa', 'bs', 'begitu', 'boleh', 'bulan', 'bhabinkamtibmas', 
                       'bhabinkantibmas', 'bhayangkara', 'bhayangkari', 'bripka', 'dan', 'dia', 'di', 'dari', 'dalam', 'dlm', 
                       'dengan', 'daripada', 'drpd', 'demi', 'dimana', 'dmn', 'dapat', 'dpt', 'dsb', 'dst', 'dll', 'dahulu',
                       'dua', 'dulunya', 'demikian', 'desa', 'ds', 'drs', 'digunakan',  'es', 'f', 'februari', 'faceshield', 
                       'gbla', 'gelora', 'gorontalo', 'gue', 'gw', 'gubernur', 'guna', 'hal', 'halo', 'haloo', 'halooo', 
                       'hanya', 'hari', 'harus', 'hujan', 'hi', 'hadi', 'tjahjanto', 'hum', 'ia', 'ini', 'itu', 'itulah', 
                       'ingin', 'jika', 'juga', 'jg', 'jumat', 'jokowi', 'januari', 'juni', 'juli', 'joko', 'kah', 'kabupaten', 
                       'kab', 'kalo', 'klo', 'kapolda', 'kapolri', 'kapolsek', 'kami', 'kamil', 'kamis', 'karena', 'krn', 
                       'kbp', 'ke', 'kecamatan', 'kec', 'kecuali', 'kelurahan', 'kel', 'kemarin', 'kmrn', 'kemenkes', 'kembali', 'kepada', 
                       'kpd', 'kereta', 'ketika', 'kita', 'kenapa', 'knp', 'kemana', 'kan', 'kok', 'kudus', 'lagi', 'lg', 
                       'lain', 'lautan', 'listyo', 'leksono', 'maka', 'maret', 'mari', 'marsekal', 'masih', 'masker', 'menurut', 
                       'msh', 'mereka', 'mrk', 'mengapa', 'melainkan', 'merupakan', 'mei', 'minggu', 'mobilitas', 'namun', 
                       'nanti', 'nggak', 'noh', 'oh', 'oleh', 'ok', 'orang', 'pada', 'pd', 'para', 'pakansari', 'pasti', 
                       'panglima', 'pemkab', 'pemerintah', 'polda', 'polisi', 'polres', 'polri', 'polsek', 'puskesmas', 'pula', 
                       'pmk', 'prabowo', 'presiden', 'rabu', 'rs', 'ridwan', 'saat', 'sabtu', 'sambil', 'sampai', 'satgas', 
                       'saja', 'saya', 'sy', 'sebab', 'sebagai', 'sebelum', 'sblm', 'sebetulnya', 'secara', 'sedangkan', 
                       'seharusnya', 'sehingga', 'sekitar', 'selagi', 'sementara', 'selain', 'seolah', 'seperti', 'seraya', 
                       'serta', 'sesuatu', 'sesudah', 'setelah', 'tadi', 'tapi', 'tp', 'tanpa', 'telah', 'tentang', 'ttg', 
                       'tentu', 'terhadap', 'terminal', 'tetapi', 'tidak', 'toh', 'tolong', 'tni', 'uea', 'untuk', 
                       'wakapol', 'wakapolda', 'walau', 'widodo', 'ya', 'yaitu', 'yakni', 'yang'])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("kamus/stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    #kata = [word for word in words if word not in list_stopwords]
    return [word for word in words if word not in list_stopwords]

df['stopwords'] = df['normalisasi'].apply(stopwords_removal) 

df.loc[0:9,['normalisasi', 'stopwords']]

,normalisasi,stopwords
0,"[kalau, penumpang, baru, saja, masuk, umur, da...","[penumpang, umur, aplikasi, pedulilindungi, ti..."
1,"[paling, wajib, scan, qr, artinya, wajib, vaks...","[wajib, scan, qr, wajib, vaksin, booster, lanj..."
2,"[semoga, saja, pink, karena, yang, sudah, bias...","[semoga, pink, kebalikannya, kai, pakai, vaksi..."
3,"[jadi, jika, belum, dapat, vaksin, booster, bl...","[vaksin, booster, kakak]"
4,"[ready, tembak, vaksin, vaksin, hapus, data, d...","[ready, tembak, vaksin, vaksin, hapus, data, d..."
5,"[pak, mohon, informasi, rs, di, medan, yang, b...","[informasi, medan, vaksin, booster]"
6,"[jangan, takut, aku, sudah, vaksin, booster]","[takut, vaksin, booster]"
7,"[sebenarnya, yang, masuk, rumah, sakit, dan, m...","[rumah, sakit, meninggal, habis, vaksinsedangk..."
8,"[saya, deh, kali, kena, covid, walaupun, deh, ...","[covid, vaksinbooster, sayang, serik, wehmeran..."
9,"[mau, tanya, dong, kalau, mau, nonton, konser,...","[nonton, konser, vaksin, booster, nonton, exo,..."


## Stemming

In [9]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

# estimation time for stemming : 10 minutes (aproximately)

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['stopwords']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print("Panjang kamus term stemming: ",len(term_dict))
print("------------------------\n")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":",term_dict[term])

print("------------------------")

# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['stemming'] = df['stopwords'].swifter.apply(get_stemmed_term)

df.loc[0:9,['stopwords', 'stemming']]

Pandas Apply:   0%|          | 0/7302 [00:00<?, ?it/s]

,stopwords,stemming
0,"[penumpang, umur, aplikasi, pedulilindungi, ti...","[tumpang, umur, aplikasi, pedulilindungi, tike..."
1,"[wajib, scan, qr, wajib, vaksin, booster, lanj...","[wajib, scan, qr, wajib, vaksin, booster, lanjut]"
2,"[semoga, pink, kebalikannya, kai, pakai, vaksi...","[moga, pink, balik, kai, pakai, vaksin, booster]"
3,"[vaksin, booster, kakak]","[vaksin, booster, kakak]"
4,"[ready, tembak, vaksin, vaksin, hapus, data, d...","[ready, tembak, vaksin, vaksin, hapus, data, d..."
5,"[informasi, medan, vaksin, booster]","[informasi, medan, vaksin, booster]"
6,"[takut, vaksin, booster]","[takut, vaksin, booster]"
7,"[rumah, sakit, meninggal, habis, vaksinsedangk...","[rumah, sakit, tinggal, habis, vaksinsedangkan..."
8,"[covid, vaksinbooster, sayang, serik, wehmeran...","[covid, vaksinbooster, sayang, serik, wehmeran..."
9,"[nonton, konser, vaksin, booster, nonton, exo,...","[nonton, konser, vaksin, booster, nonton, exo,..."


In [12]:
df.to_csv("dataset/tweets_clean.csv")
df.to_excel("dataset/tweets_clean.xlsx")

In [13]:
df.shape

(7302, 9)